# Lab 10
Bruce Cai

Use a single hidden layer with 10 units, and dropout regularization. Compare the classification performance of your model with that of linear logistic regression.

In [12]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [29]:
Default = load_data('Default').dropna()
n = Default.shape[0]
#print(Default.head(4))
Default['default'] = Default['default'].map({'Yes': 1, 'No': 0})
model = MS(Default.columns.drop('default'), intercept=False)
X = model.fit_transform(Default).to_numpy()
scaler = StandardScaler()
X = scaler.fit_transform(X)
Y = Default['default'].to_numpy()
(X_train, 
 X_test,
 Y_train,
 Y_test) = train_test_split(X,
                            Y,
                            test_size=0.2,
                            random_state=1)


In [36]:
log_reg = LogisticRegression()
log_reg.fit(X_train, Y_train)

y_log_pred = log_reg.predict(X_test)
test_accuracy_log = accuracy_score(Y_test, y_log_pred)

print("LogisticRegression accuracy is: ",test_accuracy_log)
cm = confusion_matrix(Y_test, y_log_pred)
print(cm)

LogisticRegression accuracy is:  0.974
[[1930   11]
 [  41   18]]


In [31]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(Y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(Y_test, dtype=torch.float32).unsqueeze(1)

In [ ]:
class DefaultModel(nn.Module):
    def __init__(self, input_size):
        super(DefaultModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 10),  
            nn.ReLU(),                 
            nn.Dropout(0.4)            
        )
        self.output_layer = nn.Linear(10, 1)  

    def forward(self, x):
        x = self.layer1(x)
        x = self.output_layer(x)
        return torch.sigmoid(x) 
input_size = X_train.shape[1]
model = DefaultModel(input_size)


criterion = nn.BCELoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 10/100, Loss: 0.3599
Epoch 20/100, Loss: 0.2405
Epoch 30/100, Loss: 0.1813
Epoch 40/100, Loss: 0.1538
Epoch 50/100, Loss: 0.1371
Epoch 60/100, Loss: 0.1255
Epoch 70/100, Loss: 0.1185
Epoch 80/100, Loss: 0.1116
Epoch 90/100, Loss: 0.1102
Epoch 100/100, Loss: 0.1059


In [ ]:
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test)
    y_pred_test_class = (y_pred_test >= 0.5).float()  


accuracy = (y_pred_test_class == y_test).float().mean()
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9705


The LogisticRegression accuracy is 0.974, which is slightly higher than the single layer NN model.